<a href="https://colab.research.google.com/github/ridazaneb/ohio-sdoh-dashboard/blob/main/ohio-sdoh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas openpyxl


In [4]:

import pandas as pd
import sqlite3
import os

# Load Excel data
excel_path = "2024_County_Health_Ohio_Data.xlsx"
data = pd.read_excel(excel_path, sheet_name="Additional Measure Data")

# Relevant columns
cols = [
    "Unnamed: 2", "Life Expectancy", "% Not Proficient in English", "% Female",
    "% Rural", "% Non-Hispanic White", "% Hispanic"
]
income_cols = [c for c in data.columns if 'income' in str(c).lower()]
housing_cols = [c for c in data.columns if 'housing' in str(c).lower()]
disease_cols = [c for c in data.columns if 'diabetes' in str(c).lower() or 'obesity' in str(c).lower()]
final_cols = list(set(cols + income_cols + housing_cols + disease_cols))

df = data[final_cols].rename(columns={"Unnamed: 2": "County"})
df = df.dropna(subset=["County"])
for col in df.columns:
    if col != "County":
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Save to SQLite
conn = sqlite3.connect("ohio_health_data.db")
df.to_sql("county_health", conn, if_exists="replace", index=False)
print("✅ Data saved to db/ohio_health_data.db")


✅ Data saved to db/ohio_health_data.db


In [5]:
# STEP 1: Install packages
!pip install pandas plotly statsmodels

# STEP 2: Upload the .db file
from google.colab import files
uploaded = files.upload()  # Upload ohio_health_data.db

# STEP 3: Load from SQLite
import sqlite3
import pandas as pd
import plotly.express as px
import statsmodels.api as sm

conn = sqlite3.connect("ohio_health_data.db")

# Load datasets
df_income_life = pd.read_sql("""
SELECT County, [Median Household Income], [Life Expectancy]
FROM county_health
WHERE [Median Household Income] IS NOT NULL AND [Life Expectancy] IS NOT NULL
""", conn)

df_housing_diabetes = pd.read_sql("""
SELECT County, [Severe Housing Cost Burden], [Diabetes Prevalence]
FROM county_health
WHERE [Severe Housing Cost Burden] IS NOT NULL AND [Diabetes Prevalence] IS NOT NULL
""", conn)

df_income_housing = pd.read_sql("""
SELECT County, [Median Household Income], [Severe Housing Cost Burden]
FROM county_health
WHERE [Median Household Income] IS NOT NULL AND [Severe Housing Cost Burden] IS NOT NULL
""", conn)

# STEP 4: Plot 1 — Life Expectancy vs Income
fig1 = px.scatter(df_income_life, x="Median Household Income", y="Life Expectancy",
                  text="County", trendline="ols", title="Life Expectancy vs Median Income")
fig1.show()

X1 = sm.add_constant(df_income_life["Median Household Income"])
model1 = sm.OLS(df_income_life["Life Expectancy"], X1).fit()
print(model1.summary())

# STEP 5: Plot 2 — Diabetes vs Housing Burden
fig2 = px.scatter(df_housing_diabetes, x="Severe Housing Cost Burden", y="Diabetes Prevalence",
                  text="County", trendline="ols", title="Diabetes vs Housing Burden")
fig2.show()

X2 = sm.add_constant(df_housing_diabetes["Severe Housing Cost Burden"])
model2 = sm.OLS(df_housing_diabetes["Diabetes Prevalence"], X2).fit()
print(model2.summary())

# STEP 6: Plot 3 — Income vs Housing Burden
fig3 = px.scatter(df_income_housing, x="Median Household Income", y="Severe Housing Cost Burden",
                  text="County", trendline="ols", title="Income vs Housing Burden")
fig3.show()

X3 = sm.add_constant(df_income_housing["Median Household Income"])
model3 = sm.OLS(df_income_housing["Severe Housing Cost Burden"], X3).fit()
print(model3.summary())


Saving ohio_health_data.db to ohio_health_data (1).db


                            OLS Regression Results                            
Dep. Variable:        Life Expectancy   R-squared:                       0.561
Model:                            OLS   Adj. R-squared:                  0.556
Method:                 Least Squares   F-statistic:                     110.0
Date:                Tue, 15 Apr 2025   Prob (F-statistic):           4.71e-17
Time:                        05:04:32   Log-Likelihood:                -157.68
No. Observations:                  88   AIC:                             319.4
Df Residuals:                      86   BIC:                             324.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

                             OLS Regression Results                            
Dep. Variable:     Diabetes Prevalence   R-squared:                       0.012
Model:                             OLS   Adj. R-squared:                  0.001
Method:                  Least Squares   F-statistic:                     1.061
Date:                 Tue, 15 Apr 2025   Prob (F-statistic):              0.306
Time:                         05:04:32   Log-Likelihood:                -120.81
No. Observations:                   88   AIC:                             245.6
Df Residuals:                       86   BIC:                             250.6
Df Model:                            1                                         
Covariance Type:             nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const     

                                OLS Regression Results                                
Dep. Variable:     Severe Housing Cost Burden   R-squared:                       0.001
Model:                                    OLS   Adj. R-squared:                 -0.010
Method:                         Least Squares   F-statistic:                    0.1018
Date:                        Tue, 15 Apr 2025   Prob (F-statistic):              0.751
Time:                                05:04:32   Log-Likelihood:                -957.44
No. Observations:                          88   AIC:                             1919.
Df Residuals:                              86   BIC:                             1924.
Df Model:                                   1                                         
Covariance Type:                    nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------